In [3]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression

In [107]:
# choose a list of 5 countries to start with abundant data
countries = ['India', 'Canada', 'Spain', 'Norway', 'Brazil']

In [112]:
def compute_percent_change_features(country_list, start_date, train_date, test_date):
    def percent_change(new, original):
        return (new - original) / original
    
    # store dataframes in lists
    df_list = []

    for country in countries:
        country_df = pd.read_csv(f'{country}_biodiversity_research.csv')
        country_df = country_df[country_df['Country Specific Authors'] > 0]
        country_df = country_df.reset_index(drop=True)

        # get papers from 1998 and onwards
        country_df = country_df[country_df['Publication Year'] >= 1998]
        country_df = country_df.reset_index(drop=True)
        df_list.append(country_df)

    country_train = []
    country_test = []

    for country_data in df_list:
        train_data = pd.DataFrame(dict({'Year': np.zeros(train_date - 1998 + 1), 'Paper Volume': np.zeros(train_date - 1998 + 1), 'Mean Adj. Citations': np.zeros(train_date - 1998 + 1), 'Mean Authors': np.zeros(train_date - 1998 + 1), 'Mean Orgs': np.zeros(train_date - 1998 + 1)}))
        test_data = pd.DataFrame(dict({'Year': np.zeros(test_date - train_date), 'Paper Volume': np.zeros(test_date - train_date), 'Mean Adj. Citations': np.zeros(test_date - train_date), 'Mean Authors': np.zeros(test_date - train_date), 'Mean Orgs': np.zeros(test_date - train_date)}))
        for date in range(1998, test_date + 1):
            year_data = country_data[country_data['Publication Year'] == date]
            year_data = year_data.reset_index(drop=True)

            if date <= train_date:
                train_data.iloc[date - 1998] = [date, len(year_data), year_data['Adjusted Citations'].mean(), year_data['Country Specific Authors'].mean(), year_data['Country Specific Orgs'].mean()]
            else:
                test_data.iloc[date - train_date - 1] = [date, len(year_data), year_data['Adjusted Citations'].mean(), year_data['Country Specific Authors'].mean(), year_data['Country Specific Orgs'].mean()]

        country_train.append(train_data)
        country_test.append(test_data)
    
    protected_df = pd.read_csv('protected_land_cleaned.csv')
    
    # now compute the percent changes starting from start_date (i.e. can start analysis in start_date + 1)
    
    percent_change_train = []
    percent_change_test = []
    percent_change_protected_test = []
    percent_change_protected_train = []

    for train_df in country_train:
        percent_train = pd.DataFrame(dict({'Change Volume': np.zeros(train_date - start_date), 'Change Citations': np.zeros(train_date - start_date), 'Change Authors': np.zeros(train_date - start_date), 'Change Orgs': np.zeros(train_date - start_date)}))
        for i in range(train_date - start_date):
            percent_train.iloc[i] = [percent_change(train_df.iloc[start_date - 1998 + 1 + i]['Paper Volume'], train_df.iloc[start_date - 1998 + i]['Paper Volume']), percent_change(train_df.iloc[start_date - 1998 + 1 + i]['Mean Adj. Citations'], train_df.iloc[start_date - 1998 + i]['Mean Adj. Citations']), percent_change(train_df.iloc[start_date - 1998 + 1 + i]['Mean Authors'], train_df.iloc[start_date - 1998 + i]['Mean Authors']), percent_change(train_df.iloc[start_date - 1998 + 1 + i]['Mean Orgs'], train_df.iloc[start_date - 1998 + i]['Mean Orgs'])]
        percent_change_train.append(percent_train)

    for test_df in country_test:    
        percent_test = pd.DataFrame(dict({'Change Volume': np.zeros(test_date - train_date - 1), 'Change Citations': np.zeros(test_date - train_date - 1), 'Change Authors': np.zeros(test_date - train_date - 1), 'Change Orgs': np.zeros(test_date - train_date - 1)}))
        for i in range(test_date - train_date - 1):
            percent_test.iloc[i] = [percent_change(test_df.iloc[1 + i]['Paper Volume'], test_df.iloc[0 + i]['Paper Volume']), percent_change(test_df.iloc[1 + i]['Mean Adj. Citations'], test_df.iloc[0 + i]['Mean Adj. Citations']), percent_change(test_df.iloc[1 + i]['Mean Authors'], test_df.iloc[0 + i]['Mean Authors']), percent_change(test_df.iloc[1 + i]['Mean Orgs'], test_df.iloc[0 + i]['Mean Orgs'])]
        percent_change_test.append(percent_test)

    for country in countries:
        country_protected = protected_df[protected_df['Country'] == country]
        country_protected = country_protected[country_protected['Year'] >= start_date]
        country_protected = country_protected[country_protected['Year'] <= test_date]
        country_protected = country_protected.reset_index(drop=True)

        protect_change_train = pd.DataFrame(dict({'Change Protected Percent': np.zeros(train_date - start_date)}))
        for i in range(train_date - start_date):
            protect_change_train.iloc[i] = [percent_change(country_protected.iloc[1 + i]['Value'], country_protected.iloc[0 + i]['Value'])]
        percent_change_protected_train.append(protect_change_train)

        protect_change_test = pd.DataFrame(dict({'Change Protected Percent': np.zeros(test_date - train_date - 1)}))
        for i in range(test_date - train_date - 1):
            protect_change_test.iloc[i] = [percent_change(country_protected.iloc[train_date - start_date + 2 + i]['Value'], country_protected.iloc[train_date - start_date + 1 + i]['Value'])]
        percent_change_protected_test.append(protect_change_test)
        
    return percent_change_train, percent_change_test, percent_change_protected_train, percent_change_protected_test

In [117]:
X_train_list, X_test_list, y_train_list, y_test_list = compute_percent_change_features(countries, 2008, 2015, 2021)

C:\Users\btiv\AppData\Local\Temp/ipykernel_37148/2144858103.py:9: DtypeWarning: Columns (11,15,50,52) have mixed types. Specify dtype option on import or set low_memory=False.
  country_df = pd.read_csv(f'{country}_biodiversity_research.csv')
C:\Users\btiv\AppData\Local\Temp/ipykernel_37148/2144858103.py:9: DtypeWarning: Columns (15,50) have mixed types. Specify dtype option on import or set low_memory=False.
  country_df = pd.read_csv(f'{country}_biodiversity_research.csv')
C:\Users\btiv\AppData\Local\Temp/ipykernel_37148/2144858103.py:9: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  country_df = pd.read_csv(f'{country}_biodiversity_research.csv')


In [114]:
X_train_list[0]

,Change Volume,Change Citations,Change Authors,Change Orgs
0,0.365385,-0.015368,0.201878,-0.012860
1,0.535211,3.042571,0.007597,0.092631
2,-0.045872,-0.522897,0.076308,-0.079573
3,0.163462,0.382460,-0.053136,0.010074
4,-0.066116,0.114562,0.105852,0.064146
5,0.141593,-0.216268,0.135983,0.018314
6,0.348837,0.650278,-0.141648,0.088154


In [80]:
def calc_MSE(y_pred, y_true):
    return (1/len(y_pred))*sum(((np.array(y_pred)-np.array(y_true))**2))[0]

In [91]:
# build a simple Linear Regression Model for Country 0 (Australia)

regr = LinearRegression()
regr = regr.fit(X_train_list[1], y_train_list[1])

# get MSE on the test set
calc_MSE(regr.predict(X_test_list[1]), y_test_list[1])

0.0002350805856206705

In [93]:
regr.predict(X_test_list[1])

array([[0.01557058],
       [0.02493809],
       [0.02422093],
       [0.02609117],
       [0.0279672 ]])

In [94]:
y_test_list[1]

,Change Protected Percent
0,0.004889
1,0.041158
2,0.027800
3,0.027897
4,0.000000


In [95]:
regr.coef_[0]

array([ 0.01080126, -0.01417392,  0.09594324, -0.10475879])

In [92]:
from sklearn.preprocessing import PolynomialFeatures

# choose polynomial order than minimizes MSE on test set
degrees = list(range(1, 11))

for degree in degrees:
    # polynomial feature transform
    poly = PolynomialFeatures(degree)
    X_poly_train = poly.fit_transform(X_train_list[1])
    X_poly_test = poly.transform(X_test_list[1])

    # fit a model with the new polynomial features
    poly_regr = LinearRegression()
    poly_regr = poly_regr.fit(X_poly_train, y_train_list[1])

    # get MSE on test set
    print(f'MSE of Degree {degree} polynomial: {calc_MSE(poly_regr.predict(X_poly_test), y_test_list[1])}')

MSE of Degree 1 polynomial: 0.0002350805856206707
MSE of Degree 2 polynomial: 0.0005891141585564678
MSE of Degree 3 polynomial: 0.0006578988067246081
MSE of Degree 4 polynomial: 0.0006553028384233133
MSE of Degree 5 polynomial: 0.0006563629948500312
MSE of Degree 6 polynomial: 0.0006551914489672175
MSE of Degree 7 polynomial: 0.0006551471101835943
MSE of Degree 8 polynomial: 0.0006550792187165464
MSE of Degree 9 polynomial: 0.0006550814064453103
MSE of Degree 10 polynomial: 0.0006550788118346283


In [87]:
# train the optimal degree 3 polynomial
poly = PolynomialFeatures(3)
X_poly_train = poly.fit_transform(X_train_list[0])
X_poly_test = poly.transform(X_test_list[0])

# fit a model with the new polynomial features
poly_regr = LinearRegression()
poly_regr = poly_regr.fit(X_poly_train, y_train_list[0])

print('Predictions:\n', poly_regr.predict(X_poly_test))
print('\nActual Values:\n', y_test_list[0]['Change Protected Percent'])

Predictions:
 [[ 0.03921306]
 [ 0.06903876]
 [ 0.08466675]
 [ 0.03615381]
 [-0.20262732]]

Actual Values:
 0    0.013382
1    0.005280
2    0.000147
3    0.049414
4    0.000000
Name: Change Protected Percent, dtype: float64


In [ ]:
# Next steps: Add latent variables to X_train data